# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[1]
filepath

'/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json'

In [6]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id','title','artist_id','year','duration']].values[0].tolist()
song_data

['SOUDSGM12AC9618304',
 'Insatiable (Instrumental Version)',
 'ARNTLGG11E2835DDB9',
 0,
 266.39628]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
artist_data

['ARNTLGG11E2835DDB9', 'Clp', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')
log_files[0]

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [12]:
filepath = log_files[len(log_files)-1]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-17-events.json'

In [13]:
df = pd.read_json(filepath,lines=True)
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page'] =='NextSong'] 
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


In [15]:
from datetime import datetime
df['timestamd'] = df['ts'].apply(lambda x: datetime.fromtimestamp(x/1000.0))
#df['timestamd']= pd.to_datetime(df['ts'], unit='ms')
df.head()
t = df[['timestamd']].copy()
t.head(3)

,timestamd
0,2018-11-17 00:02:24.796
1,2018-11-17 00:06:40.796
2,2018-11-17 00:10:51.796


In [16]:
# function create list data before insert
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
def convert_to_time_data(t):
    # strure time table (timestamd,hour,day,week,month,year,weekday)
    return [t,t.hour,t.day,t.isocalendar()[1],t.month,t.year,t.weekday()]
time = pd.DataFrame(columns=column_labels)   
#print(convert_to_time_data(value))
index = 0
for item, value in t['timestamd'].items():
    #print(convert_to_time_data(value))
    time.loc[len(time.index)] = convert_to_time_data(value)
    #print(value)
    #time.append(convert_to_time_data(value))
time.head(10)

,start_time,hour,day,week,month,year,weekday
0,2018-11-17 00:02:24.796,0,17,46,11,2018,5
1,2018-11-17 00:06:40.796,0,17,46,11,2018,5
2,2018-11-17 00:10:51.796,0,17,46,11,2018,5
3,2018-11-17 00:14:35.796,0,17,46,11,2018,5
4,2018-11-17 00:17:39.796,0,17,46,11,2018,5
5,2018-11-17 00:20:55.796,0,17,46,11,2018,5
6,2018-11-17 00:27:01.796,0,17,46,11,2018,5
7,2018-11-17 00:28:07.796,0,17,46,11,2018,5
8,2018-11-17 00:31:21.796,0,17,46,11,2018,5
9,2018-11-17 00:37:14.796,0,17,46,11,2018,5


In [17]:
#time_df = time.astype(int)
#df['start_time'] = pandas.to_datetime(df['date'], unit='s')
time.head(3)
#print(type(time['start_time'][0]))

,start_time,hour,day,week,month,year,weekday
0,2018-11-17 00:02:24.796,0,17,46,11,2018,5
1,2018-11-17 00:06:40.796,0,17,46,11,2018,5
2,2018-11-17 00:10:51.796,0,17,46,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time.iterrows():
    #print(i,'and',row)
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head(3)

,userId,firstName,lastName,gender,level
0,49,Chloe,Cuevas,F,paid
1,49,Chloe,Cuevas,F,paid
2,49,Chloe,Cuevas,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    #print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    #print(songid,artistid)
    # insert songplay record
    songplay_data = (row.timestamd, row.userId, row.level, str(songid), str(artistid), row.sessionId, row.location, row.userAgent)
    songplay_data
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.